
# K-Means Clustering Tingkat Kemiskinan Provinsi Indonesia (2025)

Notebook ini **menggunakan penentuan jumlah cluster secara OTOMATIS (Auto-K)**  
berdasarkan **Elbow Method dan Silhouette Score**, sehingga **K tidak ditentukan manual**.

Notebook ini dirancang agar:
- Mudah dipresentasikan
- Mudah dijelaskan ke dosen
- Konsisten dengan metodologi jurnal ilmiah

**Sumber Data**  
Badan Pusat Statistik (BPS) – Persentase Penduduk Miskin (P0)  
Provinsi Indonesia, Semester I (Maret) 2025



## Konsep Dasar Metode

### 1. K-Means Clustering
K-Means adalah metode unsupervised learning yang mengelompokkan data
berdasarkan kedekatan jarak ke pusat cluster (centroid).

Masalah utama K-Means:
> Jumlah cluster (K) harus ditentukan di awal.

Oleh karena itu, pada penelitian ini digunakan **Auto-K**.



### 2. Elbow Method
Elbow Method digunakan untuk melihat perubahan nilai **inertia**
(jumlah jarak data ke centroid).

- Jika penurunan inertia mulai melambat,
  maka titik tersebut dianggap sebagai jumlah cluster yang optimal.



### 3. Silhouette Score
Silhouette Score mengukur **kualitas pemisahan cluster**.

- Nilai mendekati **1** → cluster sangat baik
- Nilai mendekati **0** → cluster saling tumpang tindih

Jumlah cluster optimal dipilih berdasarkan **nilai Silhouette tertinggi**.


In [1]:

# Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score


In [8]:

# Load dataset
df_raw = pd.read_csv(
    "kemiskinan.csv"
)
df_raw.head()


FileNotFoundError: [Errno 2] No such file or directory: 'kemiskinan.csv'

In [ ]:

# ==============================
# DATA CLEANING
# ==============================

df = df_raw.rename(columns={
    df_raw.columns[0]: "Provinsi",
    "Unnamed: 1": "Perkotaan",
    "Unnamed: 4": "Perdesaan",
    "Unnamed: 7": "Jumlah"
})

df = df[["Provinsi", "Perkotaan", "Perdesaan", "Jumlah"]]

df = df[
    df["Provinsi"].notna() &
    (df["Provinsi"] != "INDONESIA")
].copy()

for col in ["Perkotaan", "Perdesaan", "Jumlah"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df = df.dropna().reset_index(drop=True)
df.info()



## Normalisasi Data
Data dinormalisasi menggunakan **StandardScaler**
agar setiap variabel memiliki skala yang sebanding
dan tidak mendominasi proses clustering.


In [ ]:

X = df[["Perkotaan", "Perdesaan", "Jumlah"]]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)



## Auto-K: Penentuan Jumlah Cluster Otomatis

Langkah:
1. Menghitung Elbow (Inertia)
2. Menghitung Silhouette Score
3. Memilih K dengan **Silhouette Score tertinggi**


In [ ]:

inertia = {}
silhouette_scores = {}

K_range = range(2, 7)

for k in K_range:
    model = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = model.fit_predict(X_scaled)
    inertia[k] = model.inertia_
    silhouette_scores[k] = silhouette_score(X_scaled, labels)

# Menentukan K terbaik berdasarkan Silhouette tertinggi
best_k = max(silhouette_scores, key=silhouette_scores.get)
best_k, silhouette_scores[best_k]


In [ ]:

# Visualisasi Elbow & Silhouette
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.plot(list(inertia.keys()), list(inertia.values()), marker="o")
plt.title("Elbow Method")
plt.xlabel("Jumlah Cluster")
plt.ylabel("Inertia")

plt.subplot(1,2,2)
plt.plot(list(silhouette_scores.keys()), list(silhouette_scores.values()), marker="o")
plt.title("Silhouette Score")
plt.xlabel("Jumlah Cluster")
plt.ylabel("Score")

plt.tight_layout()
plt.show()



## Hasil Auto-K
Berdasarkan evaluasi Silhouette Score,
jumlah cluster optimal yang diperoleh adalah:

> **K = 2**


In [ ]:

# K-Means final menggunakan K optimal (AUTO)
kmeans_final = KMeans(n_clusters=best_k, random_state=42, n_init=10)
df["Cluster"] = kmeans_final.fit_predict(X_scaled)

df.head()


In [ ]:

# Analisis rata-rata tiap cluster
cluster_summary = df.groupby("Cluster")[
    ["Perkotaan", "Perdesaan", "Jumlah"]
].mean()

cluster_summary



## Interpretasi (Siap Presentasi)

- **Cluster 0**  
  Provinsi dengan tingkat kemiskinan relatif rendah hingga menengah.

- **Cluster 1**  
  Provinsi dengan tingkat kemiskinan tinggi,
  terutama didominasi wilayah perdesaan.

Kesimpulan:
Auto-K memastikan bahwa jumlah cluster
dipilih secara objektif dan tidak ditentukan manual.
